In [55]:
import numpy as np
import pandas as pd

In [56]:
#read csv
players_data = pd.read_csv('league_of_graphs_players.csv')

games_players_data = pd.read_csv('games_players_data.csv')

games_data = pd.read_csv('Games_data_raw.csv')

In [57]:
def data_processing(data):
  #new cols 'division', 'rank', 'lp' full of nones
  data['division'] = np.nan
  data['rank'] = np.nan
  data['lp'] = np.nan
  
  for i in range(len(data)):
    #split rank_lp to rank, lp
    if len(data.loc[i, 'rank_lp'].split()) == 4:
      div, rank, lp, _ = data.loc[i, 'rank_lp'].split(' ')
      data.loc[i, ['division', 'rank', 'lp']] = div, rank, lp
    elif len(data.loc[i, 'rank_lp'].split()) == 3:
      div, lp, _ = data.loc[i, 'rank_lp'].split(' ')
      data.loc[i, ['division', 'rank', 'lp']] = div, None, lp
    else:
      div, rank = data.loc[i, 'rank_lp'].split(' ')
      data.loc[i, ['division', 'rank', 'lp']] = div, rank, 0
  
  romav_dict = {'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, None: None}
  data['rank'] = data['rank'].map(romav_dict)
  
  
  
  
  #split winrate to winrate, games_played. last part is rate, second last is game_played
  data[['_', 'games_won', 'winrate']] = data['winrate'].str.split(' ', expand=True)
  #filter winrate from % and ()
  data['winrate'] = data['winrate'].str.replace('%', '').str.replace('(', '').str.replace(')', '')
  #split kda to kills, death, assists
  data[['kills', 'death', 'assists']] = data['kda'].str.split('/', expand=True)
  #drop rank_lp, kda
  data = data.drop(['rank_lp', 'kda', '_'], axis=1) # Remove '_' from the list of columns to drop.
  #show first 10 lines
  return data

In [58]:
players_data = data_processing(players_data)
games_players_data = data_processing(games_players_data)

C:\Users\mfranko\AppData\Local\Temp\ipykernel_28256\2478756024.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Challenger' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['division', 'rank', 'lp']] = div, None, lp
C:\Users\mfranko\AppData\Local\Temp\ipykernel_28256\2478756024.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1248' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['division', 'rank', 'lp']] = div, None, lp
C:\Users\mfranko\AppData\Local\Temp\ipykernel_28256\2478756024.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'I' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

In [59]:
players_data.dropna(subset=['winrate'], inplace=True)

bad_names = []
for index, row in games_players_data.iterrows():
    if row['winrate'] is np.nan:
        bad_names.append(row['username'])
games_players_data.dropna(subset=['winrate'], inplace=True)

In [60]:
for name in bad_names:
    games_data = games_data[~games_data.isin([name]).any(axis=1)]

In [61]:
#export to csv
players_data.to_csv('league_of_graphs_players_filtered.csv', index=False)

games_players_data.to_csv('games_players_data_filtered.csv', index=False)

games_data.to_csv('Games_data_raw_filtered.csv', index=False)